# 文本矩阵化
以cnews.test.txt为例

In [1]:
path = './cnews.test.txt'
with open(path,'r',encoding='UTF-8') as f:
    cnews_test = f.readlines()

In [2]:
# 将test中的label取出
test_label,test_x = [],[]
for each in cnews_test:
    each0 = each.split('\t')
    test_label.append(each0[0])
    test_x.append(each0[1])

jieba分词并去停用词

In [3]:
# 载入停用词字典，对其进行去停用词
with open('./stopword.txt','r',encoding='UTF-8') as f:
    stopwords = f.readlines()
a = ''
for each in stopwords:
    a = a + ' '+each
stopwords = a.replace('\n','').split(' ')
stopwords = [each for each in stopwords if each not in ['\n','']]

In [4]:
import jieba
# 使用jieba精确分词
test_x = [[each0 for each0 in jieba.cut(each) if each0 not in stopwords] for each in test_x]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\lejon\AppData\Local\Temp\jieba.cache
Loading model cost 0.778 seconds.
Prefix dict has been built succesfully.


In [6]:
len(test_x)

10000

In [14]:
# test_x有10000个样本，取500:1500之间的1000个样本为例子
test_x1 = test_x[500:1500]
# 因为test_x1存放的是单个词语构成的list，现在将其按照' '将其为字符串形式
test_x1 = [' '.join(each) for each in test_x1] 
test_x1[0]

'华府 收官战 阿联 送 大礼   获 霸道 数据 枉 老板 惊呼 新浪 体育讯 华盛顿 奇才 主场 历经 加时赛 95 94 险胜 强敌 波士顿 凯尔特人 本赛季 主场 比赛 落幕 战 易建联 上场 27 分钟 投 10 篮板 助攻 抢断 球队 篮板王 纵观 整场 比赛 球队 替补 球员 易建联 这场 命中率 比赛 进攻 端 发挥 只能 算是 下半场 调整 改进 整体 着眼 层次 进攻 发挥 只能 算是 中规中矩 阿联 本场 比赛 依然 不乏 亮点 全场 比赛 抢下 10 篮板球 进攻 篮板 两项 数据 全队 阿联 27 19 出场 时间 轻易 发现 阿联 篮板 效率 依然 全队 全场 杰 梅因 奥尼尔 杰夫 格林 篮板球 超过 阿联 出场 时间 更是 远超 阿联 本赛季 大部分 时间 阿联 篮板球 能力 出场 时间 大幅 削减 缘故 阿联 本赛季 场均 3.7 篮板 进账 这一 成绩 位居 奇才 计算 36 分钟 理论 数据 阿联 7.7 位居 篮板 争抢 率上 阿联 前场 篮板 率 7.5% 位居 球队 后场 篮板 率为 17.2 位居 球队 总 篮板 率为 12.2 位居 球队 角度看 阿联 本赛季 篮板球 争抢 没什么 优势 阿联 证明 数据 骗人 事实上 出场 时间 少 希尔顿 阿姆斯特朗 凯文 塞拉芬 相关 数据 排 阿联 符合实际 阿联 比赛 证明 足够 表现 空间 阿联 胜任 球队 主力 篮板 手 角色 阿联 场 比赛 场 扮演 球队 替补 篮板王 角色 比赛 阿联 球队 替补 篮板王 抢下 篮板 进攻 篮板 阿联 爆发 老板 特德 利昂 西斯 直呼 阿联 变 身 篮板 野兽 阿联 确实 老板 惊呼 本场 比赛 再度 展现 出色 篮板球 意识 第一节 出场 时间 余下 三节 狂揽 篮板球 第二节 单节 抢下 篮板 进攻 篮板 第三节 再度 抓下 篮板 第四节 抢下 篮板 三节 比赛 26 分钟 阿联 抢下 常规 时间 全队 最多 10 篮板 这份 成绩 足以 告慰 老板 利昂 西斯 惊叹 阿联 勇猛 主帅 菲利普 桑德斯 全方位 信任 关键 加时赛 阿联 安德雷 布 莱切 次 犯规 毕业 出场 时间 临时工 身份 自然 阿联 略显 无所适从 送出 至关重要 助攻 阿联 无论是 进攻 防守 还 找到 感觉 实际上 阿联 资格 早 出场 贾 维尔 麦基

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1,2),min_df=0.2, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)#
# 参数：ngram_range=(1,2)可理解为滑窗，设置(1,2),意思是取临近的两个词语计算；min_df=0.2，max_df=0.9，表示只取词频率在0.1-0.9的词语
# 之所以设置了这一部，主要为降低经过TF-IDF后的维数或者特征
tfidf_matrix = tfidf_vec.fit_transform(test_x1)
X_matrix  = tfidf_matrix.toarray()

In [16]:
# 将test中前1000个样本中的label取出
X_label = test_label[500:1500]
# 将label值转换为数值类型
char2num = dict(zip(list(set(X_label)),range(len(set(X_label)))))
X_labelnum = []
for each in X_label:
    X_labelnum.append(char2num[each])

In [25]:
X_matrix.shape

(1000, 39)

In [41]:
# 现计算样本label与样本特征之间一一的互信息
X_matrix_T = X_matrix.T
X_MI = {} # 用于存储X_matrix特征与label之间的互信息值
from sklearn import metrics as mr
import numpy as np
X_labels = np.array(X_labelnum)
for i in range(X_matrix_T.shape[0]):
    X_MI[i] = mr.mutual_info_score(X_labels,X_matrix_T[i])

In [42]:
X_MI

{0: 0.24619826572194375,
 1: 0.14522977235188136,
 2: 0.14734647651463229,
 3: 0.16740174049361115,
 4: 0.16633030232276233,
 5: 0.19369112636560065,
 6: 0.5428524740217371,
 7: 0.2731074874796676,
 8: 0.3488441926564956,
 9: 0.2897688674350848,
 10: 0.31808366843365193,
 11: 0.1422161015337544,
 12: 0.2935651532009065,
 13: 0.16263006881211767,
 14: 0.3794654626012728,
 15: 0.20706098165646547,
 16: 0.27186005501996524,
 17: 0.452993759900236,
 18: 0.19204408298638884,
 19: 0.5851166818798166,
 20: 0.5428524740217371,
 21: 0.30016036017557496,
 22: 0.43714680147354773,
 23: 0.15959821836588517,
 24: 0.1836438452099457,
 25: 0.43928617857707036,
 26: 0.15283062967068475,
 27: 0.1698194131383874,
 28: 0.2511756223994213,
 29: 0.29619120665417087,
 30: 0.21030153310342578,
 31: 0.17932399572937224,
 32: 0.22561620724647047,
 33: 0.21063663539398503,
 34: 0.18325480335078095,
 35: 0.17008334106832496,
 36: 0.18087980084327804,
 37: 0.16778248299242965,
 38: 0.18394306130519347}

In [49]:
# 筛选特征，根据互信息的大小进行从大到小排序，留下前20的特征
X_MIF = [each for each in X_MI if X_MI[each]>=sorted(X_MI.values())[10]]
print('选择的特征对应列序号：',X_MIF)
X_fea = [X_matrix[each] for each in X_MIF]
X_matrix_fea = np.array(X_fea).T

选择的特征对应列序号： [0, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38]
